# Deploying a function (without an explicit "model") on Verta

Within Verta, a "Model" can be any arbitrary function: a traditional ML model (e.g., sklearn, PyTorch, TF, etc); a function (e.g., squaring a number, making a DB function etc.); or a mixture of the above (e.g., pre-processing code, a DB call, and then a model application (See more here: )

This notebook provides an example of how to deploy any function on Verta.

## 0. Imports

In [1]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [2]:
# Ensure credentials are set up, if not, use below
import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST'] = 

from verta import Client

client = Client(os.environ['VERTA_HOST'])

## 1. Define data transformation

In [3]:
# Suppose we have a cubic function, 3x^3 + 2x + 5 that we want to implement
def cubic_transform(x):
    return 3*x*x*x + 2*x + 5

## 1. b Wrap data transform in a class deriving from VertaModelBase

In [4]:
from verta.registry import VertaModelBase
class CubicFunction(VertaModelBase):
    def __init__(self, artifacts=None):
        pass
    
    def predict(self, input_data):
        output_data = []
        for input_data_point in input_data:
            output_data_point = cubic_transform(input_data_point)
            output_data.append(output_data_point)
        return output_data

## 2. Define a registered model for deployment

In [5]:
registered_model = client.get_or_create_registered_model(
    name="cubic-function", labels=["data-transform"])

In [6]:
from verta.environment import Python

model_version = registered_model.create_standard_model(
    CubicFunction,
    environment=Python(requirements=[]),
    name="v1",
)

## 3. Deploy model to endpoint

In [7]:
cubic_function_endpoint = client.get_or_create_endpoint("cubic")
cubic_function_endpoint.update(model_version, wait=True)

In [8]:
deployed_model = cubic_function_endpoint.get_deployed_model()
deployed_model.predict([3, 0])